## <p style="text-align: center;">BA108_2122_TMD2_CA_2</p>
### <p style="text-align: center;"> Details </p>
#### <p style="text-align: center;">Name: Sunil Judhistira Gauda</p>
#### <p style="text-align: center;">Student ID : 10595858 </p>
##### <p style="text-align: center;">Github : https://github.com/sunil3092/BA108_2122_TMD2_CA2 </p>

In [1]:
import requests as req
from bs4 import BeautifulSoup
import re
import time
import sqlite3
import pandas as pd
import uuid

In [2]:
def manage_tags(html, tags):
    soup = BeautifulSoup(html, "html.parser")

    for page_data in soup(tags):
        page_data.decompose()

    return ''.join(soup.stripped_strings)

In [3]:
top_rated_list = []

In [4]:
page_number = [1,2,3,4,5]


In [5]:
for num in page_number:
     url = "https://www.swiggy.com/city/mumbai/top-rated-collection?page=" + str(num)
     request = req.get(url)
     soup = BeautifulSoup(request.content)
     div = soup.find_all("div", {"class" : "_3FR5S"})
     time.sleep(10)
     for i in range( len(div)):
          top_rated = {}
          top_rated["name"] = div[i].find("div", {"class" : "nA6kb"}).text
          top_rated["type"] = div[i].find("div", {"class" : "_1gURR"}).text
          top_rated["rating"] = float(re.sub('\D','',div[i].find("div", {"class" : "_9uwBC"}).text))
          stringValue = div[i].find("div", {"class" : "_3Mn31"}).text
          split_string = stringValue.split("•")
          top_rated["deliviery_time"] = int(re.sub('\D','',split_string[1]))
          top_rated["price_for_two"] = int(re.sub('\D','',split_string[2]))
          discount = div[i].find("span", {"class" : "sNAfh"})
          if discount is not None:
               discount = discount.text
               discount = discount.split("|")
               if len(discount) == 1:
                    top_rated["discount"] = 0
                    top_rated["discount_code"] = discount
               elif len(discount) > 0:
                    top_rated["discount"] = int(re.sub('\D','', discount[0]))
                    top_rated["discount_code"] = re.sub(" ","",discount[1].replace("Use ", ""))

          else:
               top_rated["discount"] = 0
               top_rated["discount_code"] = ""
          top_rated_list.append(top_rated)

In [6]:
top_list_df = pd.DataFrame(top_rated_list)
top_list_df['discountId'] = top_list_df[["discount_code"]].sum(axis=1).map(hash)

In [7]:
discount_df = top_list_df[["discount_code", "discount", "discountId"]]

In [8]:
discount_df = discount_df.drop_duplicates()

In [9]:
discount_df = discount_df[discount_df["discount_code"] != '']

In [10]:
discount_df

,discount_code,discount,discountId
0,WELCOME50,50,-396299740802975704
5,TRYNEW,60,6270345446514787261
10,STEALDEAL,60,-8402913390663284998
11,TRYNEW,50,6270345446514787261
71,SPECIALS,50,8448648663248268164


In [11]:
def db_manager():
    conn = conn = sqlite3.connect('swiggytop.db')
    return conn

In [12]:
def write_data_to_db(df,table_name, conn, if_exists_val = 'replace', index_val = False):
    df.to_sql(name=table_name, con=conn, if_exists = if_exists_val, index = index_val )

In [13]:
conn_obj = db_manager()

In [15]:
top_list_df = top_list_df.drop(["discount", "discount_code"], axis=1)
write_data_to_db(top_list_df,"SWIGGYTOP", conn_obj, 'replace', False)

In [16]:
write_data_to_db(discount_df,"DISCOUNT", conn_obj, 'replace', False)

In [17]:
top_list_df

,name,type,rating,deliviery_time,price_for_two,discountId
0,TBHC,"Beverages, Pastas, American, Snacks",50.0,64,750,-396299740802975704
1,The Neapolitan Oven,"Pizzas, Salads",50.0,70,750,-396299740802975704
2,Jiaozi - The Asian Kitchen,"Asian, Chinese",50.0,67,450,-396299740802975704
3,Sagarika Restaurant,"Indian, Chinese",50.0,50,300,0
4,GoodDo- The Vegan Eatery,"Fast Food, North Indian, Beverages",50.0,49,300,-396299740802975704
...,...,...,...,...,...,...
75,Nothing But Chicken,"Salads, Fast Food, Raw meats",46.0,29,400,-396299740802975704
76,Churolls & Co.,"Desserts, Beverages",45.0,63,500,-396299740802975704
77,Jeevan,"North Indian, Chinese, South Indian",45.0,63,550,-396299740802975704
78,Aditi Fast Food,"Street Food, North Indian, Chinese, South Indi...",45.0,70,500,-396299740802975704


In [19]:
discount_df

,discount_code,discount,discountId
0,WELCOME50,50,-396299740802975704
5,TRYNEW,60,6270345446514787261
10,STEALDEAL,60,-8402913390663284998
11,TRYNEW,50,6270345446514787261
71,SPECIALS,50,8448648663248268164
